In [39]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from bs4 import BeautforifulSoup
from urllib.request import urlopen

In [34]:
def get_cases():
    base = "https://www.justice.gov.il"
    prev_links = None
    links = []
    n=1
    ret = set()
    while prev_links != links:
        url = f"{base}/Units/StateAttorney/Criminal/Pages/Conditional-Order.aspx?WPID=WPQ8&PN={n}"
        soup = BeautifulSoup(urlopen(url).read().decode("utf8"), 'html.parser')
        prev_links = links
        links = [base + str(link.get("href")) for link in soup.findAll("a") if link.get_text()=="להרחבה"]
        ret=ret | set(links)
        n+=1
    return list(ret)

def get_case_details(url):
    soup = BeautifulSoup(urlopen(url).read().decode("utf8"), 'html.parser')
    table_body = soup.find("div", attrs={"class": "gridDivs"})
    rows = [(tr.find("div", attrs={"class": "TitleDiv"}).get_text(), tr.find("div", attrs={"class": "GridOrangContent"}).get_text(),)
            for tr in table_body.find_all('div', attrs={"class": "DataItemGrid"})]
    ret = {k.strip(": \r\n\t"): v.strip(": \r\n\t") for k,v in rows}
    return ret

In [ ]:
case_links = get_cases()
print (len(case_links))

In [40]:
error_count = 0
with open("conditional_order.json",'w') as f:
    f.write("[\n")
    first = True
    for url in tqdm(case_links):
        if not first:
            f.write("\n,\n")
        try:
            json.dump(get_case_details(url),f, indent=4)
            first = False
        except:
            print ("Error with "+url)
            error_count +=1
            first = True
    f.write("\n]")
print (f"number of errors: {error_count}")

Error with https://www.justice.gov.ilמספר



In [48]:
with open("conditional_order.json", 'r') as f:
    conditional_order = json.load(f)
df = pd.DataFrame(conditional_order).set_index("מספר תיק")
df.head()

,הוראות החיקוק שפורטו בהסדר,יחידה,נימוקים משתנים לסגירת התיק בהסדר,תיאור העובדות המהוות עבירה שבהן הודה החשוד,תנאי ההסדר
מספר תיק,,,,,
3373/15,החזקת טובין שסומנו או יובאו לישראל שלא כדין – ...,פרקליטות מחוז דרום (פלילי),"החשוד הודה בעובדות המהוות עבירה, לחשוד אין עבר...","בתאריך 30.1.15, החזיק החשוד, לשם מסחר, בדוכן ב...",קנס בסך 2900 ₪ והשמדת פריטים שנתפסו.
7937/15,א. גניבה - עבירה לפי סעיף 384 לחוק העונשין התש...,פרקליטות מחוז תל-אביב (פלילי),"היקף המרמה אינו מאוד גבוה, שכן מדובר על צ'ק אח...","ביום 09/06/15 או סמוך לאחריו, מצא החשוד 12 המח...","א. תשלום לאוצר המדינה בסך 2,000 ₪.ב. פיצוי למת..."
4625/15,השארת ילד בלא השגחה ברשלנות – עבירה לפי סעיף 3...,פרקליטות מחוז חיפה (פלילי),החשודות הודו בעובדות המהוות עבירהלחשודות אין ע...,החשודות הינן סייעת וגננת בגן בצפון הארץ. ביום ...,"פיצוי כספי להורי הילד בסך 1,000 ₪תשלום לאוצר ה..."
3613/16,ניסיון למעשה מגונה – עבירה לפי סעיף 25 + 348 ...,פרקליטות מחוז צפון (פלילי),א. החשוד הודה בעובדות המהוות עבירה.\r\nב. לחשו...,"1. במועדים הרלבנטיים לתלונת המתלוננת, עבד החשו...","-פיצוי ע""ס 2,500 ₪ למתלוננת.\r\n-העברת מכתב הת..."
9213/13,מעשי פזיזית ורשלנות (אי נקיטת צעדי זהירות בחיה...,פרקליטות מחוז תל-אביב (פלילי),הנאשמת הודתה בעובדות המהוות עברה ולקחה אחריות ...,"הנאשמת הינה בעלת כלב מסוג ""רועה בלגי"". בשלהי ח...","1. תשלום לאוצר המדינה בסך 800 ש""ח אשר שולם עד ..."


In [49]:
df.to_csv("conditional_order.csv")